In [ ]:
!python --version

In [ ]:
import tensorflow as tf
#import tensorflow_datasets as tfds
print("TensorFlow version:", tf.__version__)
#print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

In [ ]:
# import du dataset
import pandas as pd
import seaborn as sns
import numpy as np

In [ ]:
df = pd.read_csv('../../dataset/edible_mushrooms.csv')

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
fig, ax = plt.subplots(figsize=(12,3))
ax = sns.countplot(data = df, x = 'mixed', alpha = 0.7)

plt.setp(ax.get_xticklabels(), rotation=30, ha="right")
ax.set_xlabel('mixed')
ax.set_ylabel('nombre de photos')
ax.set_title('nombre de photos par species pour les champignons comestibles');

In [ ]:
max_size = df['order'].value_counts().max()
max_size = min(max_size, 5000)

In [ ]:
for index, value in df['order'].value_counts().items():
    print(f"{index}: {value}")

# Generating data

In [ ]:
train_path='../../dataset/mixed/train'
test_path='../../dataset/mixed/test'

In [ ]:
from tensorflow.keras.applications.mobilenet import preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
train_data_generator = ImageDataGenerator(
    preprocessing_function = preprocess_input)

In [ ]:
train_generator = train_data_generator.flow_from_directory(
    directory = train_path,
    class_mode = "sparse",
    target_size = (224,224), batch_size = 16)

In [ ]:
test_data_generator = ImageDataGenerator(
    preprocessing_function = preprocess_input)

In [ ]:
test_generator = test_data_generator.flow_from_directory(
    directory = test_path,
    class_mode = "sparse",
    target_size = (224,224), batch_size = 16)

# transfer learning on Mobile Net

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications.mobilenet import MobileNet
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, GlobalAveragePooling2D

n_class= 38#df['order'].unique().shape[0]

base_model = MobileNet(weights = 'imagenet', include_top = False, input_shape=(224,224,3))

# Get the output of the base model
base_output = base_model.output

for layer in base_model.layers:
    layer.trainable = False

# Add your custom layers
x = GlobalAveragePooling2D()(base_output)  # Flatten layer
x = Dense(1024, activation='relu')(x)  # Dense layer
x = Dropout(rate=0.3)(x)
x = Dense(n_class, activation='softmax')(x)  # Dense layer

# Create a new model that includes the base model and your custom layers
model_mobile = Model(inputs=base_model.input, outputs=x)

In [ ]:
model_mobile.compile(optimizer='adam', 
               loss='sparse_categorical_crossentropy', 
               metrics=['acc'])
model_mobile.summary()

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard
import os
from datetime import datetime

os.environ["CUDA_VISIBLE_DEVICES"]="1"  # specify which GPU(s) to be used

# Create an early stopping callback
early_stopping = EarlyStopping(monitor='acc', patience=5, min_delta = 0.001)

# saving history in a CSV file
filename='model_mobile_mixed_history.csv'
history_logger=tf.keras.callbacks.CSVLogger(filename, separator=",", append=True)

# Define the Keras TensorBoard callback.
logdir="logs/fit/" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = TensorBoard(log_dir=logdir)

# Train the model.

model_mobile_history = model_mobile.fit(
    train_generator,
    epochs = 20,
    validation_data=test_generator,
    callbacks=[early_stopping, history_logger, tensorboard_callback]
)

In [ ]:
# Save the model
model_mobile.save('./model_mobile_mixed_checkpoint')

In [ ]:
import pandas as pd
model_mobile_history = pd.read_csv('model_mobile_mixed_history.csv')

In [ ]:
import matplotlib.pyplot as plt

# Assume `history` is the history object returned by the `fit` method
# Plot the accuracy and validation accuracy
plt.plot(model_mobile_history['acc'])
plt.plot(model_mobile_history['val_acc'])
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

In [ ]:
from tensorflow.keras.models import load_model

model_mobile = load_model('./model_mobile_mixed_checkpoint')

In [ ]:
from sklearn.metrics import confusion_matrix

y_pred_proba = model_mobile.predict(test_generator)
y_pred = np.argmax(y_pred_proba, axis=1)

y_true = test_generator.classes

In [ ]:
from sklearn.metrics import confusion_matrix

y_pred_proba = model_mobile.predict(test_generator)
y_pred = np.argmax(y_pred_proba, axis=1)

y_true = test_generator.classes

In [ ]:
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(y_true, y_pred, normalize = 'true')

# Display the confusion matrix
plt.figure(figsize=(24, 24)) 
sns.heatmap(cm, annot=True, cmap='Blues')
plt.ylabel('True label')
plt.xlabel('Predicted label')
plt.show()

In [ ]:
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(y_true, y_pred, normalize = 'true')

# Display the confusion matrix
plt.figure(figsize=(24, 24)) 
sns.heatmap(cm, annot=True, cmap='Blues')
plt.ylabel('True label')
plt.xlabel('Predicted label')
plt.show()

In [ ]:
%tensorboard --logdir logs

## adding data augmentation

In [ ]:
train_data_generator = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=20,     # randomly rotate images in the range (degrees, 0 to 180)
    width_shift_range=0.3, # randomly shift images horizontally (fraction of total width)
    height_shift_range=0.3,# randomly shift images vertically (fraction of total height)
    horizontal_flip=True,  # randomly flip images horizontally
    vertical_flip = True,
    zoom_range=0.4         # Randomly zoom image 
)

In [ ]:
train_generator = train_data_generator.flow_from_directory(
    directory = train_path,
    class_mode = "sparse",
    target_size = (224,224), batch_size = 16)

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard
import os
from datetime import datetime

os.environ["CUDA_VISIBLE_DEVICES"]="1"  # specify which GPU(s) to be used

# Create an early stopping callback
early_stopping = EarlyStopping(monitor='acc', patience=5, min_delta = 0.001)

# saving history in a CSV file
filename='model_mobile_mixed_aug_history.csv'
history_logger=tf.keras.callbacks.CSVLogger(filename, separator=",", append=True)

# Define the Keras TensorBoard callback.
logdir="logs/fit/" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = TensorBoard(log_dir=logdir)

# Train the model.

model_mobile_history = model_mobile.fit(
    train_generator,
    epochs = 50,
    validation_data=test_generator,
    callbacks=[early_stopping, history_logger, tensorboard_callback]
)

In [ ]:
# Save the model
model_mobile.save('./model_mobile_mixed_aug_checkpoint')

In [ ]:
from sklearn.metrics import confusion_matrix

y_pred_proba = model_mobile.predict(test_generator)
y_pred = np.argmax(y_pred_proba, axis=1)

y_true = test_generator.classes

In [ ]:
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(y_true, y_pred, normalize = 'true')

# Display the confusion matrix
plt.figure(figsize=(24, 24)) 
sns.heatmap(cm, annot=True, cmap='Blues')
plt.ylabel('True label')
plt.xlabel('Predicted label')
plt.show()

In [ ]:
import pandas as pd
model_mobile_history = pd.read_csv('model_mobile_mixed_aug_history.csv')

In [ ]:
import matplotlib.pyplot as plt

# Assume `history` is the history object returned by the `fit` method
# Plot the accuracy and validation accuracy
plt.plot(model_mobile_history['acc'][0:50])
plt.plot(model_mobile_history['val_acc'][0:50])
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()